# Домашнее задание №6 - Нейросети в обработке текста

- Самостоятельно обучить классификатор текстов на примере 20newsgroups

- На примере 20 newsgroups попробовать разные параметры для сверток для классификации текстов

In [67]:
import random, re, glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from functools import lru_cache
from tqdm.notebook import tqdm

In [63]:
fields = ['newsgroup', 'document_id', 'from', 'subject', 'archive-name', 'last-modified']

def parse_file(path):
    path = path.replace('\\', '/')
    newsgroup=path.rsplit('/', 1)[1].rsplit('.', 1)[0]
    print(f"Читаем {path}, newsgroup {newsgroup}")
    res = []
    header=False
    with open(path, 'r') as fd:
        for line in fd:
            m = re.match(fr"({'|'.join(fields)}):\s*(\S.*)", line, re.I)
            if m:
                if not header:
                    res.append({ 'text': '', **dict((f, '') for f in fields) })
                    res[-1]['newsgroup'] = newsgroup
                    header = True
                res[-1][m.group(1).lower()] = m.group(2)
            else:
                if header:
                    if line.strip() and not re.match(fr"[\w\-]+:.*", line, re.I):
                        header = False
                if not header:
                    res[-1]['text'] += line
    return res

@lru_cache()
def parse_directory(path):
    return pd.DataFrame([ r for f in glob.glob(path + '/*.txt') for r in parse_file(f) ])      


In [74]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

from collections import Counter

class Dataset(torch.utils.data.Dataset):
    
    
    def __init__(self, data_path):
        df = parse_directory(data_path)

        self.labels_i2w = df['newsgroup'].unique() 
        self.labels_w2i = dict( (w, i) for i, w in enumerate(self.labels_i2w) )
        self.labels = df['newsgroup'].apply(lambda ng: self.labels_w2i[ng]).tolist()
        
        sw = set(get_stop_words("en"))
        exclude = set(punctuation)
        morpher = MorphAnalyzer()

        def preprocess(txt):
            txt = ''.join(c for c in txt if c not in exclude)
            txt = txt.lower()
            return [morpher.parse(word)[0].normal_form for word in txt.split() if word and word not in sw] # удалить пустые сразу

        print('Подготовка текста ...')
        texts = [preprocess(txt) for txt in tqdm(df['text'].tolist())]
        
        print('Преобразование в индексы ...')
        words = [w for w, c in Counter(w for txt in texts for w in txt).items() if c > 10]
        
        self.words_i2w = ['', '__UNK__'] + words
        self.words_w2i = w2i = dict( (w, i) for i, w in enumerate(self.words_i2w) )
        
        max_length = min(max([len(txt) for txt in texts]), 1024)
        
        self.length = max_length
       
        texts = [[w2i.get(w, w2i['__UNK__']) for w in txt] for txt in texts]
        self.texts = [ torch.tensor(txt[:max_length] + [0] * (max_length - len(txt))) for txt in texts ]

        assert(len(self.texts) == len(self.labels))
        
        print('Dataset подготовлен')
        
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

ds = Dataset('./20newsgroups')

Подготовка текста ...


  0%|          | 0/39675 [00:00<?, ?it/s]

Преобразование в индексы ...
Dataset подготовлен


In [80]:
from sklearn.model_selection import train_test_split 

ds_train, ds_val = train_test_split(ds, test_size=0.1, random_state=7)

In [90]:
def evaluate(net, ds):
    dl = torch.utils.data.DataLoader(ds, batch_size=128, shuffle=False)
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.eval()
    total_loss = 0
    count = 0
    matches = 0
    for X, y in dl:
        count += len(y)
        preds = net(X)
        total_loss += float(loss(preds, y)) # очень важно преобразовать во float здесь, иначе утекает память !!!
        pred_classes = torch.argmax(preds, dim=1)
        matches += int(sum(pred_classes == y))
    return float(total_loss/count), float(matches/count), count


def train_and_test(net, ds_train, ds_val, optimizer_class=torch.optim.Adam, n_epochs=10, lr=0.01, report_on=-1, batch_size=1024):
    
    print('========================================================')
   
    params = [ p for p in net.parameters() if p.requires_grad]
    print('Число обучаемых параметров', len(params))
    optimizer = optimizer_class(params, lr=0.01)
    print(' Оптимизатор: ', optimizer)

    def print_results():
        for title, ds in [
#                ('Тренировочный', ds_train), # слишком долго для resnet 
                ('Валидационный', ds_val)
            ]:
            r = evaluate(net, ds)
            print(f"    {title} набор: кросс-энтропия: {r[0]:.2f}, доля совпадений: {r[1]*100:.1f}%")
        return r
    
    print(' До обучения:')
    print_results()
    
    dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size)
    criterion = nn.CrossEntropyLoss()    

    net.train()
    report_on = max(len(dl_train) if report_on == -1 else report_on, 1)
    for epoch in range(n_epochs):
        print('--------- Эпоха ', epoch, '/', n_epochs, ' ------------------')
        running_loss = 0.0
        for i, data in enumerate(tqdm(dl_train)):
            inputs, labels = data[0], data[1]

            # обнуляем градиент
            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # выводим статистику о процессе обучения
            running_loss += loss.item()
            if (i+1) % report_on == 0:    # печатаем каждые report_on mini-batches
                print('Выучено батчей : %5d; loss: %.3f' % (i + 1, running_loss / report_on))
                running_loss = 0.0
                
    print(' Обучение закончено!')
    print(' После обучения:')
    return print_results()

In [84]:
class Net(nn.Module):
    def __init__(self, num_classes, vocab_size, embedding_dim = 128, linear_dim = 256, kernel_size=3):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.conv = nn.Conv1d(embedding_dim, linear_dim, kernel_size=kernel_size)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(linear_dim, num_classes)
        
    def forward(self, x):        
        output = self.embedding(x)
        #                       B  F  L         
        output = output.permute(0, 2, 1)
        output = self.conv(output)
        output = self.relu(output)
        output = torch.max(output, axis=2).values
        output = self.linear(output)
        
        return output   

In [83]:
net = Net(20, len(ds.words_i2w))
train_and_test(net, ds_train, ds_val)

Число обучаемых параметров 5
 Оптимизатор:  Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)
 До обучения
    Валидационный набор: кросс-энтропия: 3.32, доля совпадений: 5.3%
--------- Эпоха  0 / 10  ------------------


  0%|          | 0/35 [00:00<?, ?it/s]

Выучено батчей :    35; loss: 2.538
--------- Эпоха  1 / 10  ------------------


  0%|          | 0/35 [00:00<?, ?it/s]

Выучено батчей :    35; loss: 0.505
--------- Эпоха  2 / 10  ------------------


...

Выучено батчей :    35; loss: 0.013
--------- Эпоха  8 / 10  ------------------


  0%|          | 0/35 [00:00<?, ?it/s]

Выучено батчей :    35; loss: 0.012
--------- Эпоха  9 / 10  ------------------


  0%|          | 0/35 [00:00<?, ?it/s]

Выучено батчей :    35; loss: 0.013
 Обучение закончено!
 После обучения
    Валидационный набор: кросс-энтропия: 0.07, доля совпадений: 98.2%


In [85]:
results = [{ 'embedding_dim': 128, 'linear_dim': 256, 'kernel_size': 3, 'net': net, 'result': evaluate(net, ds_val) }] # сохраним обученный net, остальное в цикле ниже


In [87]:
def create_and_test(embedding_dim=128, linear_dim=256, kernel_size=3):
    print('########################################################################')
    print(f"embedding_dim = {embedding_dim}, linear_dim = {linear_dim}, kernel_size = {kernel_size}")
    net = Net(20, len(ds.words_i2w), kernel_size=2)
    r = train_and_test(net, ds_train, ds_val)
    results.append({ 'embedding_dim': embedding_dim, 'linear_dim': linear_dim, 'kernel_size': kernel_size, 'net': net, 'result': r })    
    
    
for kernel_size in [2, 5, 7]:
    create_and_test(kernel_size=kernel_size)

for linear_dim in [128, 256, 512]:
    create_and_test(linear_dim=linear_dim)

for embedding_dim in [ 64, 256 ]:
    create_and_test(embedding_dim=embedding_dim)

########################################################################
embedding_dim = 128, linear_dim = 256, kernel_size = 2
Число обучаемых параметров 5
 Оптимизатор:  Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)
 До обучения
    Валидационный набор: кросс-энтропия: 3.33, доля совпадений: 4.7%
--------- Эпоха  0 / 10  ------------------


  0%|          | 0/35 [00:00<?, ?it/s]

Выучено батчей :    35; loss: 2.455
--------- Эпоха  1 / 10  ------------------


  0%|          | 0/35 [00:00<?, ?it/s]

Выучено батчей :    35; loss: 0.573
--------- Эпоха  2 / 10  ------------------


...

Выучено батчей :    35; loss: 0.012
--------- Эпоха  8 / 10  ------------------


  0%|          | 0/35 [00:00<?, ?it/s]

Выучено батчей :    35; loss: 0.011
--------- Эпоха  9 / 10  ------------------


  0%|          | 0/35 [00:00<?, ?it/s]

Выучено батчей :    35; loss: 0.012
 Обучение закончено!
 После обучения
    Валидационный набор: кросс-энтропия: 0.07, доля совпадений: 98.3%


In [91]:
for r in tqdm(results): # ошибка в train_and_test была исправлена позже, результат не возвращался
    r['result'] = evaluate(r['net'], ds_val)
pd.DataFrame(results)

  0%|          | 0/9 [00:00<?, ?it/s]

,embedding_dim,linear_dim,kernel_size,net,result
0,128,256,3,"Net(\n (embedding): Embedding(32254, 128, pad...","(0.0724949883897939, 0.9816028225806451, 3968)"
1,128,256,2,"Net(\n (embedding): Embedding(32254, 128, pad...","(0.07741514596367075, 0.983366935483871, 3968)"
2,128,256,5,"Net(\n (embedding): Embedding(32254, 128, pad...","(0.07222905139168424, 0.9826108870967742, 3968)"
3,128,256,7,"Net(\n (embedding): Embedding(32254, 128, pad...","(0.07564804668448144, 0.9831149193548387, 3968)"
4,128,128,3,"Net(\n (embedding): Embedding(32254, 128, pad...","(0.06760406659375276, 0.9831149193548387, 3968)"
5,128,256,3,"Net(\n (embedding): Embedding(32254, 128, pad...","(0.06324990069673907, 0.9838709677419355, 3968)"
6,128,512,3,"Net(\n (embedding): Embedding(32254, 128, pad...","(0.06815058317395949, 0.9838709677419355, 3968)"
7,64,256,3,"Net(\n (embedding): Embedding(32254, 128, pad...","(0.06880994937232425, 0.9848790322580645, 3968)"
8,256,256,3,"Net(\n (embedding): Embedding(32254, 128, pad...","(0.07026325794117104, 0.983366935483871, 3968)"


In [92]:
create_and_test(embedding_dim=64, linear_dim=256, kernel_size=2)

########################################################################
embedding_dim = 64, linear_dim = 256, kernel_size = 2
Число обучаемых параметров 5
 Оптимизатор:  Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)
 До обучения:
    Валидационный набор: кросс-энтропия: 3.24, доля совпадений: 4.9%
--------- Эпоха  0 / 10  ------------------


  0%|          | 0/35 [00:00<?, ?it/s]

Выучено батчей :    35; loss: 2.374
--------- Эпоха  1 / 10  ------------------


  0%|          | 0/35 [00:00<?, ?it/s]

Выучено батчей :    35; loss: 0.544
--------- Эпоха  2 / 10  ------------------


...

Выучено батчей :    35; loss: 0.011
--------- Эпоха  8 / 10  ------------------


  0%|          | 0/35 [00:00<?, ?it/s]

Выучено батчей :    35; loss: 0.011
--------- Эпоха  9 / 10  ------------------


  0%|          | 0/35 [00:00<?, ?it/s]

Выучено батчей :    35; loss: 0.011
 Обучение закончено!
 После обучения:
    Валидационный набор: кросс-энтропия: 0.07, доля совпадений: 98.4%


In [93]:
create_and_test(embedding_dim=64, linear_dim=256, kernel_size=7)

########################################################################
embedding_dim = 64, linear_dim = 256, kernel_size = 7
Число обучаемых параметров 5
 Оптимизатор:  Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)
 До обучения:
    Валидационный набор: кросс-энтропия: 3.51, доля совпадений: 4.5%
--------- Эпоха  0 / 10  ------------------


  0%|          | 0/35 [00:00<?, ?it/s]

Выучено батчей :    35; loss: 2.430
--------- Эпоха  1 / 10  ------------------


  0%|          | 0/35 [00:00<?, ?it/s]

Выучено батчей :    35; loss: 0.547
--------- Эпоха  2 / 10  ------------------


...

Выучено батчей :    35; loss: 0.011
--------- Эпоха  8 / 10  ------------------


  0%|          | 0/35 [00:00<?, ?it/s]

Выучено батчей :    35; loss: 0.011
--------- Эпоха  9 / 10  ------------------


  0%|          | 0/35 [00:00<?, ?it/s]

Выучено батчей :    35; loss: 0.011
 Обучение закончено!
 После обучения:
    Валидационный набор: кросс-энтропия: 0.08, доля совпадений: 98.3%


In [95]:
df = pd.DataFrame(results)
df['loss'] = df['result'].apply(lambda r: r[0])
df['matches'] = df['result'].apply(lambda r: r[1])
df

,embedding_dim,linear_dim,kernel_size,net,result,loss,matches
0,128,256,3,"Net(\n (embedding): Embedding(32254, 128, pad...","(0.0724949883897939, 0.9816028225806451, 3968)",0.072495,0.981603
1,128,256,2,"Net(\n (embedding): Embedding(32254, 128, pad...","(0.07741514596367075, 0.983366935483871, 3968)",0.077415,0.983367
2,128,256,5,"Net(\n (embedding): Embedding(32254, 128, pad...","(0.07222905139168424, 0.9826108870967742, 3968)",0.072229,0.982611
3,128,256,7,"Net(\n (embedding): Embedding(32254, 128, pad...","(0.07564804668448144, 0.9831149193548387, 3968)",0.075648,0.983115
4,128,128,3,"Net(\n (embedding): Embedding(32254, 128, pad...","(0.06760406659375276, 0.9831149193548387, 3968)",0.067604,0.983115
5,128,256,3,"Net(\n (embedding): Embedding(32254, 128, pad...","(0.06324990069673907, 0.9838709677419355, 3968)",0.063250,0.983871
6,128,512,3,"Net(\n (embedding): Embedding(32254, 128, pad...","(0.06815058317395949, 0.9838709677419355, 3968)",0.068151,0.983871
7,64,256,3,"Net(\n (embedding): Embedding(32254, 128, pad...","(0.06880994937232425, 0.9848790322580645, 3968)",0.068810,0.984879
8,256,256,3,"Net(\n (embedding): Embedding(32254, 128, pad...","(0.07026325794117104, 0.983366935483871, 3968)",0.070263,0.983367
9,64,256,2,"Net(\n (embedding): Embedding(32254, 128, pad...","(0.07365286535012626, 0.9836189516129032, 3968)",0.073653,0.983619


In [97]:
df.iloc[df['matches'].idxmax()] 

embedding_dim                                                   64
linear_dim                                                     256
kernel_size                                                      3
net              Net(\n  (embedding): Embedding(32254, 128, pad...
result             (0.06880994937232425, 0.9848790322580645, 3968)
loss                                                       0.06881
matches                                                   0.984879
Name: 7, dtype: object